斐波那契数列( Fibonacci Sequence)，又称黄金分割数列，  
指的是这样一个数列：1,1,2,3,5,8,13,21, ... ,  
这个数列从第三项开始，每一项都等于前两项之和。求数列第n项。

In [10]:
def fibonacci(n):
    if n <= 1:
        return 1
    return fibonacci(n-1) + fibonacci(n-2)

In [5]:
def fibonacci(n, cache=None):
    if cache is None:
        cache = {}
    
    if n in cache:
        return cache[n]
    
    if n <= 1:
        return 1
    cache[n] = fibonacci(n-1, cache) + fibonacci(n-2, cache)
    return cache[n]

In [6]:
print(fibonacci(50))

20365011074


In [7]:
def memo(func):
    cache = {}
    def wrap(*args):
        if args not in cache:
            cache[args] = func(*args)
        return cache[args]
    return wrap

In [14]:
@memo
def fibonacci(n):
    if n <= 1:
        return 1
    return fibonacci(n-1) + fibonacci(n-2)

print(fibonacci(50))

20365011074


一个共有十个台阶的楼梯，从下面走到上面，一次只能迈1-3个台阶，  
并且不能后退，走完这个楼梯共有多少种方法。

In [3]:
def climb(n, steps):
    count = 0
    if n == 0:
        count = 1
    elif n > 0:
        for step in steps:
            count += climb(n-step, steps)
    return count

In [13]:
@memo
def climb(n, steps):
    count = 0
    if n == 0:
        count = 1
    elif n > 0:
        for step in steps:
            count += climb(n-step, steps)
    return count

print(climb(10, (1,2,3)))

274


## 如何为被装饰的函数保存元数据?

In [15]:
def f(a):
    ''' f function '''
    return a * 2

In [16]:
f.__name__

'f'

In [17]:
g = f
g.__name__

'f'

In [18]:
f.__doc__

' f function '

In [19]:
f.__module__

'__main__'

In [20]:
f.__defaults__

In [21]:
def f(a, b=1, c=[]):
    print(a, b, c)

In [22]:
f.__defaults__  # 保存函数的默认参数

(1, [])

In [23]:
f.__defaults__[1].append('abc')
f(100)
# 在默认参数中尽量不要使用可变对象

100 1 ['abc']


In [24]:
f.__closure__

In [25]:
def f():
    a = 2
    return lambda k: a ** k

In [26]:
g = f()

In [27]:
g.__closure__

(<cell at 0x000001616A8B74C8: int object at 0x00000000636C6C30>,)

In [28]:
c = g.__closure__[0]

In [30]:
c.cell_contents

2

In [31]:
def mydecorator(func):
    def wrapper(*args, **kargs):
        '''wrapper function'''
        print('In wrapper')
        func(*args, **kargs)
    return wrapper

In [32]:
@mydecorator
def example():
    '''example function'''
    print('In example')

In [33]:
print(example.__name__)

wrapper


In [34]:
print(example.__doc__)

wrapper function


In [38]:
def mydecorator_new(func):
    def wrapper(*args, **kargs):
        '''wrapper function'''
        print('In wrapper')
        func(*args, **kargs)
    wrapper.__name__ = func.__name__
    return wrapper

In [39]:
@mydecorator_new
def example():
    '''example function'''
    print('In example')
print(example.__name__)
print(example.__doc__)

example
wrapper function


上面的方式不优雅

使用标准库**functools**中的装饰器**wraps**装饰内部包裹函数，  
可以指定将原函数的某些属性，更新到包裹函数上面。

In [50]:
from functools import *

def mydecorator_tools(func):
    def wrapper(*args, **kargs):
        '''wrapper function'''
        print('In wrapper')
        func(*args, **kargs)
#     update_wrapper(wrapper, func, ('__name__', '__doc__'), ('__dict__',))
#     update_wrapper(wrapper, func, WRAPPER_ASSIGNMENTS, WRAPPER_UPDATES)
    update_wrapper(wrapper, func)
    return wrapper

@mydecorator_tools
def example():
    '''example function'''
    print('In example')
print(example.__name__)
print(example.__doc__)

example
example function


In [46]:
print(WRAPPER_ASSIGNMENTS)
print(WRAPPER_UPDATES)

('__module__', '__name__', '__qualname__', '__doc__', '__annotations__')
('__dict__',)


In [49]:
def mydecorator_wraps(func):
    @wraps(func)
    def wrapper(*args, **kargs):
        '''wrapper function'''
        print('In wrapper')
        func(*args, **kargs)
    return wrapper

@mydecorator_wraps
def example():
    '''example function'''
    print('In example')
print(example.__name__)
print(example.__doc__)

example
example function


## 如何定义带参数的装饰器

带参数的装饰器，也就是根据参数定制化一个装饰器，可以看成生产装饰器的工厂，  
每次调用`typeassert`，返回一个特定的装饰器，然后用它去修饰其它函数。

In [52]:
from inspect import signature

def f(a, b, c=1): pass
sig = signature(f)
sig.parameters

mappingproxy({'a': <Parameter "a">,
              'b': <Parameter "b">,
              'c': <Parameter "c=1">})

In [53]:
a = sig.parameters['a']
a.name

'a'

In [54]:
a.kind

<_ParameterKind.POSITIONAL_OR_KEYWORD: 1>

In [56]:
a.default

inspect._empty

In [57]:
a = sig.parameters['c']

In [58]:
a.default

1

In [59]:
bargs = sig.bind(str, int, int)

In [60]:
bargs.arguments

OrderedDict([('a', str), ('b', int), ('c', int)])

In [61]:
bargs.arguments['a']

str

In [62]:
bargs.arguments['b']

int

In [63]:
sig.bind_partial(str)

<BoundArguments (a=<class 'str'>)>

In [64]:
from inspect import signature

def typeassert(*ty_args, **ty_kargs):
    def decorator(func):
        # func -> a,b
        # d = {'a':int, 'b':str}
        sig = signature(func)
        btypes = sig.bind_partial(*ty_args, **ty_kargs).arguments
        def wrapper(*args, **kargs):
            # arg in d, instance(arg, d[arg])
            for name, obj in sig.bind(*args, **kargs).arguments.items():
                if name in btypes:
                    if not isinstance(obj, btypes[name]):
                        raise TypeError(f'"{name}" must be "{btypes[name]}"')
            return func(*args, **kargs)
        return wrapper
    return decorator

@typeassert(int, str, list)
def f(a, b, c):
    print(a, b, c)
    
f(1, 'abc', [1,2,3])
f(1, 2, [1,2,3])

1 abc [1, 2, 3]


TypeError: "b" must be "<class 'str'>"

## 如何实现属性可修改的函数装饰器

为分析程序内哪些函数执行时间开销较大，我们定义一个带timeout参数的函数装饰器。装饰功能如下：

1. 统计被装饰函数单词调用运行时间.
2. 时间大于参数timeout的，将此次函数调用记录到log日志中。
3. 运行时可修改timeout的值

In [68]:
from functools import wraps
import time, logging
from random import randint

def warn(timeout):
    def decorator(func):
        def wrapper(*args, **kargs):
            start = time.time()
            res = func(*args, **kargs)
            used = time.time() - start
            if used > timeout:
                msg = f'{func.__name__}:{used} > {timeout}'
                logging.warn(msg)
            return res
        return wrapper
    return decorator

@warn(1.5)
def test():
    print('In test')
    while randint(0, 1):
        time.sleep(0.5)

for _ in range(30):
    test()

In test
In test
In test
In test
In test
In test


C:\Anaconda\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  del sys.path[0]


In test
In test
In test
In test
In test
In test


In test
In test
In test


In test
In test
In test
In test
In test
In test
In test
In test


In test
In test
In test
In test


In test
In test
In test


In [69]:
from functools import wraps
import time, logging
from random import randint

def warn(timeout):
    def decorator(func):
        def wrapper(*args, **kargs):
            start = time.time()
            res = func(*args, **kargs)
            used = time.time() - start
            if used > timeout:
                msg = f'{func.__name__}:{used} > {timeout}'
                logging.warn(msg)
            return res
        def setTimeout(k):
            nonlocal timeout
            timeout = k
        wrapper.setTimeout = setTimeout
        return wrapper
    return decorator

@warn(1.5)
def test():
    print('In test')
    while randint(0, 1):
        time.sleep(0.5)

for _ in range(30):
    test()
test.setTimeout(1)
for _ in range(30):
    test()

In test
In test
In test
In test
In test
In test
In test
In test
In test
In test
In test
In test
In test
In test
In test
In test
In test
In test
In test
In test
In test


C:\Anaconda\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  del sys.path[0]


In test
In test
In test


In test
In test
In test
In test
In test
In test
In test
In test
In test
In test
In test
In test


In test


...

In test
In test
In test
In test


In test


In test
In test
In test


In test
In test
In test
In test


In test
In test
